# Price an FX Forward using an LSEG-defined curve

This notebook explains how to price an FX Forward using an FX Forward curve defined by LSEG.

## Start by importing the required libraries

In [1]:
from lseg_analytics.market_data import fx_forward_curves
from lseg_analytics.instruments import fx_forwards
from lseg_analytics_basic_client.models import RelativeAdjustableDate, ReferenceDate

## Load an FX Forward curve
To load the curve, you need to provide the space where to load the curve from and the name of the curve. This example loads an LSEG-defined curve, so it loads the curve from the 'LSEG' space. You can also load a curve of your own from the 'HOME' space. 

To see the possible curve names, place your cursor after the equals sign and trigger the autocompletion using the _Trigger Suggest_ command (`Ctrl`/`Cmd` + `i`).

In [2]:
# Load the EUR/GBP FX Forward curve
eurgbp_curve = fx_forward_curves.load(space="LSEG", name="EUR_GBP_FxForward")

## Create an FX instrument
To create an FX instrument you need to provide an 'FxForwardDefinition'. At a minimum this requires the 'quoted_currency', 'base_currency', 'payer' and 'receiver', but you can also define the 'start_date', 'end_date', 'rate', 'deal_amount' and 'contra_amount'.

In [3]:
eurgbp_fxfwd = fx_forwards.FxForward(
    definition=fx_forwards.FxForwardDefinition(
        quoted_currency="GBP",
        base_currency="EUR",
        rate=fx_forwards.FxRate(value=0.86),
        end_date=RelativeAdjustableDate(
            tenor="3M", reference_date=ReferenceDate.START_DATE
        ),
        start_date=RelativeAdjustableDate(
            tenor="1M", reference_date=ReferenceDate.SPOT_DATE
        ),
        deal_amount=1000000,
        payer="Party1",
        receiver="Party2",
    )
)
eurgbp_fxfwd.save(name="myforward", space="HOME")

True

## Define the market data used to price the FX Forward
Here we need to add the curve to the market_data that will be used to price the FX Forward. 

In [4]:
market_data = fx_forwards.MarketData()
market_data.fx_forward_curves = [fx_forwards.FxForwardCurveChoice(reference="EUR_GBP_FxForward")]

## Price it using the Curve
The 'price' method uses market_data containing the curve which was added in the previous step. In addition to this you can set further parameters including the 'valuation_date' and other preferences. All pricing parameters are optional and will default to a value if not selected. 

In [5]:
pricing_response = eurgbp_fxfwd.price(
    market_data=market_data,
    pricing_preferences=fx_forwards.FxPricingParameters(
        valuation_date="2024-02-11",
        ignore_reference_currency_holidays=True, 
        reference_currency="EUR",
    ),
)

## Display the response values
Print the description of the response to see the valuation date, start date and end date in the following format.
```javascript
{'valuationDate': '2024-02-11', 'startDate': {'unAdjusted': '2024-03-13', 'adjusted': '2024-03-13', 'dateMovingConvention': 'NextBusinessDay', 'referenceDate': 'SpotDate', 'tenor': '1M'}, 'endDate': {'unAdjusted': '2024-06-13', 'adjusted': '2024-06-13', 'dateMovingConvention': 'NextBusinessDay', 'referenceDate': 'StartDate', 'tenor': '3M'}}
```

As part of the pricing analysis, see the below example for the possible values that can be printed. 

In [6]:
# Display the pricing analysis
print(pricing_response.analytics.description)
print("\t Deal amount:", pricing_response.analytics.pricing_analysis.deal_amount)
print("\t Countra amount:", pricing_response.analytics.pricing_analysis.contra_amount)
print("\t FX Spot - Bid:", pricing_response.analytics.pricing_analysis.fx_spot.bid)
print("\t FX Spot - Ask:", pricing_response.analytics.pricing_analysis.fx_spot.ask)
print(
    "\t FX Swaps Ccy1 - Bid:",
    pricing_response.analytics.pricing_analysis.fx_swaps_ccy1.bid,
)
print(
    "\t FX Swaps Ccy1 - Ask:",
    pricing_response.analytics.pricing_analysis.fx_swaps_ccy1.ask,
)
print(
    "\t FX Swaps Ccy2 - Bid:",
    pricing_response.analytics.pricing_analysis.fx_swaps_ccy2.bid,
)
print(
    "\t FX Swaps Ccy2 - Ask:",
    pricing_response.analytics.pricing_analysis.fx_swaps_ccy2.ask,
)
print(
    "\t FX Swaps Ccy1 Ccy2 - Bid:",
    pricing_response.analytics.pricing_analysis.fx_swaps_ccy1_ccy2.bid,
)
print(
    "\t FX Swaps Ccy1 Ccy2 - Ask:",
    pricing_response.analytics.pricing_analysis.fx_swaps_ccy1_ccy2.ask,
)
print(
    "\t FX Outright Ccy1 Ccy2 - Bid:",
    pricing_response.analytics.pricing_analysis.fx_outright_ccy1_ccy2.bid,
)
print(
    "\t FX Outright Ccy1 Ccy2 - Ask:",
    pricing_response.analytics.pricing_analysis.fx_outright_ccy1_ccy2.ask,
)

{'valuationDate': '2024-02-11', 'startDate': {'unAdjusted': '2024-03-13', 'adjusted': '2024-03-13', 'dateMovingConvention': 'NextBusinessDay', 'referenceDate': 'SpotDate', 'tenor': '1M'}, 'endDate': {'unAdjusted': '2024-06-13', 'adjusted': '2024-06-13', 'dateMovingConvention': 'NextBusinessDay', 'referenceDate': 'StartDate', 'tenor': '3M'}}
	 Deal amount: 1000000.0
	 Countra amount: 860000.0
	 FX Spot - Bid: 0.854
	 FX Spot - Ask: 0.8544
	 FX Swaps Ccy1 - Bid: 0.0
	 FX Swaps Ccy1 - Ask: 0.0
	 FX Swaps Ccy2 - Bid: 28.44999999999986
	 FX Swaps Ccy2 - Ask: 28.86999999999973
	 FX Swaps Ccy1 Ccy2 - Bid: 28.449999999999864
	 FX Swaps Ccy1 Ccy2 - Ask: 28.869999999999727
	 FX Outright Ccy1 Ccy2 - Bid: 0.857688
	 FX Outright Ccy1 Ccy2 - Ask: 0.858124


In [7]:
fx_forwards.delete(name="myforward", space="HOME")

True